In [85]:
import pandas as pd
import shutil
from pathlib import Path
from lxml import html
import re
def extract_standardized_report_data(tree, report_title):
    """
    Extract report data from HTML tree and convert to standardized pandas DataFrame.
    
    Creates a consistent DataFrame structure regardless of the source report type:
    - customer_name: Name of the customer
    - item_name: Name/description of the item (covenant or tickler)
    - required_value: Required value for the item
    - actual_value: Actual value (if available)
    - period_date: Period/item date
    - due_date: Due date
    - days_past_due: Days past due (if available)
    - interval: Reporting interval (if available)
    - comments: Comments field
    - report_type: Type of source report
    - report_date: Date the report was created
    
    Args:
        tree: lxml HTML tree object
        report_title: Title of the report for context
        
    Returns:
        pandas.DataFrame or None if no data found
    """
    try:
        # Extract report creation date
        report_date = None
        date_elements = tree.xpath("//span[@class='date']")
        if date_elements:
            date_text = date_elements[0].text_content()
            # Extract date from "Report Creation Date : MM/DD/YYYY HH:MM:SS"
            date_match = re.search(r'(\d{2}/\d{2}/\d{4})', date_text)
            if date_match:
                report_date = date_match.group(1)
        
        # Find customer sections - each customer has their own table
        customer_sections = tree.xpath("//tr[contains(@class, '') or not(@class)]//span[contains(@class, 'SummaryTitle clientnameSmall') and contains(text(), 'Customer Name')]")
        
        all_report_data = []
        
        for customer_span in customer_sections:
            # Extract customer name
            customer_text = customer_span.text_content()
            customer_name = customer_text.replace('Customer Name :', '').strip()
            
            # Find the covenant table for this customer
            # Look for the uk-table uk-table-striped table that follows this customer name
            customer_row = customer_span
            current_element = customer_span
            
            # Traverse up to find the containing TD, then look for the table
            while current_element is not None:
                if current_element.tag == 'td':
                    break
                current_element = current_element.getparent()
            
            if current_element is not None:
                covenant_tables = current_element.xpath(".//table[@class='uk-table uk-table-striped']")
                
                for table in covenant_tables:
                    # Extract table data
                    rows = table.xpath(".//tr")
                    if len(rows) < 2:  # Need at least header + 1 data row
                        continue
                    
                    # Get headers
                    headers = []
                    header_row = rows[0]
                    for th in header_row.xpath(".//th"):
                        headers.append(th.text_content().strip())
                    
                    # Process data rows
                    for row in rows[1:]:
                        cells = row.xpath(".//td")
                        if not cells:
                            continue
                        
                        row_data = []
                        for cell in cells:
                            cell_text = cell.text_content().strip()
                            row_data.append(cell_text)
                        
                        if not any(cell.strip() for cell in row_data):  # Skip empty rows
                            continue
                        
                        # Create standardized record
                        item_record = {
                            'customer_name': customer_name,
                            'item_name': '',
                            'required_value': '',
                            'actual_value': '',
                            'period_date': '',
                            'due_date': '',
                            'days_past_due': '',
                            'interval': '',
                            'comments': '',
                            'report_type': report_title,
                            'report_date': report_date
                        }
                        
                        # Map fields based on headers and available data
                        for i, header in enumerate(headers):
                            if i < len(row_data):
                                value = row_data[i]
                                header_lower = header.lower()
                                
                                if ('covenant' in header_lower or 'tickler' in header_lower) and ('name' in header_lower or 'item' in header_lower):
                                    item_record['item_name'] = value
                                elif 'required' in header_lower:
                                    item_record['required_value'] = value
                                elif 'actual' in header_lower:
                                    item_record['actual_value'] = value
                                elif 'period' in header_lower or 'item date' in header_lower:
                                    item_record['period_date'] = value
                                elif 'due date' in header_lower:
                                    item_record['due_date'] = value
                                elif 'days past due' in header_lower:
                                    item_record['days_past_due'] = value
                                elif 'interval' in header_lower:
                                    item_record['interval'] = value
                                elif 'comment' in header_lower:
                                    item_record['comments'] = value
                        
                        all_report_data.append(item_record)
        
        if not all_report_data:
            print(f"No report data found for: {report_title}")
            return None
        
        # Create DataFrame
        df = pd.DataFrame(all_report_data)
        print(f"Extracted {len(df)} records for {len(df['customer_name'].unique())} customers")
        return df
        
    except Exception as e:
        print(f"Error extracting standardized report data: {str(e)}")
        return None
def extract_table_data(tree, report_title):
    """
    Legacy function - Extract table data from HTML tree and convert to pandas DataFrame.
    
    Args:
        tree: lxml HTML tree object
        report_title: Title of the report for context
        
    Returns:
        pandas.DataFrame or None if no data found
    """
    try:
        # Look for the main data table - typically has class "uk-table uk-table-striped"
        main_tables = tree.xpath("//table[@class='uk-table uk-table-striped']")
        
        if not main_tables:
            print(f"No main data table found for: {report_title}")
            return None
        
        main_table = main_tables[0]
        
        # Extract headers
        headers = []
        header_rows = main_table.xpath(".//tr[1]")
        if header_rows:
            for th in header_rows[0].xpath(".//th"):
                headers.append(th.text_content().strip())
        
        # Extract data rows
        data_rows = []
        all_rows = main_table.xpath(".//tr")[1:]  # Skip header row
        
        for row in all_rows:
            row_data = []
            for td in row.xpath(".//td"):
                # Extract text content from td
                cell_text = td.text_content().strip()
                row_data.append(cell_text)
            
            if row_data and any(cell.strip() for cell in row_data):  # Only add non-empty rows
                data_rows.append(row_data)
        
        if not data_rows:
            print(f"No data rows found for: {report_title}")
            return None
        
        # Create DataFrame
        if headers and len(headers) > 0:
            # Ensure all rows have the same number of columns as headers
            max_cols = len(headers)
            normalized_rows = []
            for row in data_rows:
                # Pad or truncate row to match header length
                normalized_row = row[:max_cols] + [''] * (max_cols - len(row))
                normalized_rows.append(normalized_row)
            
            df = pd.DataFrame(normalized_rows, columns=headers)
        else:
            # Fallback: create DataFrame without headers
            df = pd.DataFrame(data_rows)
        
        print(f"Extracted {len(df)} rows with {len(df.columns)} columns")
        return df
        
    except Exception as e:
        print(f"Error extracting table data: {str(e)}")
        return None
def process_xls_files():
    """
    Process up to 5 HTML files (saved as .xls) from assets folder and return dict of standardized dataframes.
    
    Returns:
        dict: Dictionary with dataframe names as keys and standardized pandas DataFrames as values
              All DataFrames have consistent columns regardless of source report type
    """
    
    assets_folder = Path('./assets')
    archive_folder = Path('./assets/archive')
    
    # Create folders if they don't exist
    assets_folder.mkdir(exist_ok=True)
    archive_folder.mkdir(exist_ok=True)
    
    # Find all .xls files (which are actually HTML files)
    excel_files = list(assets_folder.glob('*.xls'))
    
    # Validate file count and types
    assert len(excel_files) <= 5, f"Found {len(excel_files)} .xls files, maximum is 5"
    all_files = [f for f in assets_folder.iterdir() if f.is_file() and not f.name.startswith('.')]
    non_excel = [f for f in all_files if not f.name.endswith('.xls')]
    assert len(non_excel) == 0, f"Found non-.xls files: {[f.name for f in non_excel]}"
    
    # Mapping of report title patterns to dataframe keys
    df_mappings = {
        'covenants coming due within the next 365 days': 'covenants_coming_due_365',
        'covenants 1 or more days past due': 'covenants_past_due',
        'covenants 1 or more days in default': 'covenants_in_default',
        'ticklers coming due within 365 days': 'ticklers_coming_due_365',
        'ticklers 1 or more days past due': 'ticklers_past_due',
    }
    
    dataframes = {}
    
    # Process each HTML file (saved as .xls)
    for html_file in excel_files:
        print(f"Processing: {html_file.name}")
        
        try:
            # Read HTML content
            with open(html_file, 'r', encoding='utf-8') as f:
                html_content = f.read()
            
            # Parse HTML with lxml
            tree = html.fromstring(html_content)
            
            # Extract report title from the clientnameheading class
            title_elements = tree.xpath("//span[@class='clientnameheading']")
            if title_elements:
                report_title = title_elements[0].text_content().lower().strip()
                print(f"Report title: '{report_title}'")
                
                # Find matching dataframe key
                df_key = None
                for pattern, key in df_mappings.items():
                    if pattern.lower() in report_title:
                        df_key = key
                        break
                
                if df_key:
                    # Extract standardized report data
                    df = extract_standardized_report_data(tree, report_title)
                    if df is not None:
                        dataframes[df_key] = df
                        print(f"Mapped to: {df_key}")
                    else:
                        print(f"No report data found for: {report_title}")
                else:
                    print(f"No mapping found for report title: '{report_title}'")
            else:
                print(f"No report title found in: {html_file.name}")
        
        except Exception as e:
            print(f"Error processing {html_file.name}: {str(e)}")
        
        # Move to archive
        archive_dest = archive_folder / html_file.name
        shutil.move(str(html_file), str(archive_dest))
        print(f"Moved to archive: {html_file.name}")
    
    print(f"Processed {len(dataframes)} dataframes: {list(dataframes.keys())}")
    return dataframes
# Usage example:
# files = process_xls_files()
# 
# # Access individual report DataFrames (each with standardized columns)
# if 'covenants_past_due' in files:
#     df_covenants_pd = files['covenants_past_due'].copy()
# if 'covenants_coming_due_365' in files:
#     df_covenants_coming_due = files['covenants_coming_due_365'].copy()
# if 'covenants_in_default' in files:
#     df_covenants_default = files['covenants_in_default'].copy()
# if 'ticklers_coming_due_365' in files:
#     df_ticklers_coming_due = files['ticklers_coming_due_365'].copy()
#     
#     # Example queries on standardized data:
#     # Get all items for a specific customer
#     customer_items = df_covenants_pd[df_covenants_pd['customer_name'] == "customer name"]
#     
#     # Get all past due items
#     past_due = df_covenants_pd[df_covenants_pd['days_past_due'] != '']
#     
#     # Get items by type
#     coming_due_covenants = df_covenants_coming_due[df_covenants_coming_due['report_type'].str.contains('coming due', case=False, na=False)]




In [86]:
files = process_xls_files()

Processed 0 dataframes: []


In [87]:
# files.keys()

In [88]:
covenants_in_default = files['covenants_in_default'].copy()
covenants_in_default

KeyError: 'covenants_in_default'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_name   384 non-null    object
 1   item_name       384 non-null    object
 2   required_value  384 non-null    object
 3   actual_value    384 non-null    object
 4   period_date     384 non-null    object
 5   due_date        384 non-null    object
 6   days_past_due   384 non-null    object
 7   interval        384 non-null    object
 8   comments        384 non-null    object
 9   report_type     384 non-null    object
 10  report_date     384 non-null    object
dtypes: object(11)
memory usage: 33.1+ KB


,customer_name,item_name,required_value,actual_value,period_date,due_date,days_past_due,interval,comments,report_type,report_date
0,12 VENTURA DRIVE REALTY TRUST,LOC 30 Day Clean-up or Clean Down,30.00,,12/31/2025,04/30/2026,,,,covenants coming due within the next 365 days,07/07/2025
1,1276 BALD HILL RD LLC,Maximum LTV of XXX%,70.00,,12/31/2025,02/14/2026,,,,covenants coming due within the next 365 days,07/07/2025
2,1276 BALD HILL RD LLC,Pre-Distribution DSC,1.25,,12/31/2025,04/30/2026,,,,covenants coming due within the next 365 days,07/07/2025
3,1276 BALD HILL RD LLC,Post-Distribution DSC,1.00,,12/31/2025,04/30/2026,,,,covenants coming due within the next 365 days,07/07/2025
4,136 HUTTLESTON AVENUE REALTY TRUST,Maximum LTV of XXX%,85.00,,12/31/2025,04/30/2026,,,,covenants coming due within the next 365 days,07/07/2025
...,...,...,...,...,...,...,...,...,...,...,...
585,WYCHMERE HARBOR FUNCTIONS LP,Pre-Distribution DSC,1.25,,12/31/2025,04/30/2026,,,,covenants coming due within the next 365 days,07/07/2025
586,WYCHMERE HARBOR FUNCTIONS LP,Maximum LTV of XXX%,65.00,,12/31/2025,04/30/2026,,,,covenants coming due within the next 365 days,07/07/2025
587,WYCHMERE HARBOR FUNCTIONS LP,Pre-Distribution DSC,1.25,,12/31/2024,10/15/2025,,,,covenants coming due within the next 365 days,07/07/2025
588,WYCHMERE HARBOR FUNCTIONS LP,Maximum LTV of XXX%,65.00,,12/31/2024,10/15/2025,,,,covenants coming due within the next 365 days,07/07/2025


In [37]:
import src.fetch_cocc_data

In [38]:
cocc_data = src.fetch_cocc_data.fetch_data()

In [39]:
cocc_data = cocc_data['wh_acctcommon'].copy()

In [40]:
cocc_data

,ownersortname,loanofficer,acctofficer
0,CHERJOHN REALTY LLC,THOMAS D. KELLY,None
1,"HARTLING, DEBORAH J.",None,FRANK P. WILHELM
2,"MIGLIACCI, TODD M.",None,JUSTIN A. JEFFREY
3,"STRIDES BEHAVIORAL SERVICES, LLC",SBLC LOAN OFFICER,None
4,"TROWBRIDGE, BRENDA M.",None,MICHAEL A. HEY
...,...,...,...
89778,"POLLACK, RALPH P.",None,JUSTIN A. JEFFREY
89779,"SUNDERLAND, JAMIE M.",None,RICHARD J. CLARK
89780,"FORESMAN, ROSS J.",MARIA A. RODRIGUES,None
89781,"NAGLE, ELLA M.",None,MICHAEL A. HEY


In [50]:
# Function to get mode, handling cases where there might be multiple modes
def get_mode(series):
    series_clean = series.dropna()
    if len(series_clean) == 0:
        return None
    
    # Get unique values first
    unique_values = pd.Series(series_clean.unique())
    mode_result = unique_values.mode()
    
    # Return first mode if multiple modes exist
    return mode_result.iloc[0] if len(mode_result) > 0 else None
# Group and calculate mode
cocc_data_grouped = cocc_data.groupby('ownersortname').agg({
    'loanofficer': get_mode,
    'acctofficer': get_mode
}).reset_index()

cocc_data_grouped = cocc_data_grouped.rename(columns={
    'ownersortname':'customer_name',
    'loanofficer':'Loan Officer',
    'acctofficer':'Deposit Officer',
}).copy()
cocc_data_grouped

,customer_name,Loan Officer,Deposit Officer
0,"., CLAUDETTE",None,MOBOLAJI OMISORE
1,1-3 PROSPECT STREET CONDOMINIUM TRUST,None,MICHAEL A. HEY
2,100 PERRY STREET REAL ESTATE TRUST,None,MARLENE C. LIRA
3,100 WESTMINSTER PARTNERS LLC,None,MOBOLAJI OMISORE
4,1000 NEW STATE HIGHWAY INDUSTRIAL LLC,None,JOHN G. DUGGAN
...,...,...,...
51241,"ZWEBEN-KELLEY, KELLY A.",MARIA A. RODRIGUES,None
51242,"ZWICKER, DEAN A.",None,AMY M. BRIGGS
51243,"ZWICKER, DONALD E. JR",None,JACQUELINE A. THEIS
51244,"ZYBERT, KEITH F.",MARLENE R. BRAGANCA,None


In [51]:
cocc_data_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51246 entries, 0 to 51245
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   customer_name    51246 non-null  object
 1   Loan Officer     21588 non-null  object
 2   Deposit Officer  33518 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [ ]:
def merge_with_mode(df_dict, cocc_data_grouped):
    """
    Take in the dictionary of dataframes and append the mode of the loan officer and acct officer

    Only applies to active/dorm/non-performing accounts
    """
    merged_dict = {}
    for key, df in df_dict.items():
        merged_df = df.merge(cocc_data_grouped, on='customer_name', how='left')
        
        # handling dtypes
        date_fields = ['period_date','due_date','report_date']
        for field in date_fields:
            merged_df[field] = pd.to_datetime(merged_df[field])

        merged_df = merged_df.sort_values(by='period_date', ascending=True)

        merged_dict[key] = merged_df

    return merged_dict


In [78]:
cleaned_dict = merge_with_mode(files, cocc_data_grouped)

In [79]:
cleaned_dict.keys()

dict_keys(['ticklers_coming_due_365', 'covenants_in_default', 'ticklers_past_due', 'covenants_past_due', 'covenants_coming_due_365'])

In [80]:
ticklers_past_due = cleaned_dict['ticklers_past_due'].copy()
ticklers_past_due

,customer_name,item_name,required_value,actual_value,period_date,due_date,days_past_due,interval,comments,report_type,report_date,Loan Officer,Deposit Officer
0,02908 HOLDCO LLC,CPA Prepared F/S,,,2024-12-31,2025-04-30,68,Annually,,ticklers 1 or more days past due,2025-07-07,NaN,NaN
1,"113 BELLINGHAM, LLC",Corporate Tax Return,,,2024-12-31,2025-04-30,68,Annually,,ticklers 1 or more days past due,2025-07-07,LAURA A. STACK,LAURA A. STACK
2,117 METRO LLC,CPA Prepared F/S,,,2024-12-31,2025-04-30,68,Annually,,ticklers 1 or more days past due,2025-07-07,NaN,NaN
3,12 VENTURA DRIVE REALTY TRUST,Corporate Tax Return,,,2024-12-31,2025-04-30,68,Annually,,ticklers 1 or more days past due,2025-07-07,ANDREW RODRIGUES,None
4,123 EAST REALTY CORP,Corporate Tax Return,,,2024-12-31,2025-04-30,68,Annually,,ticklers 1 or more days past due,2025-07-07,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,ZEITERION REALTY MM LLC,Corporate Tax Return,,,2024-12-31,2025-04-30,68,Annually,BTR Form-990 for 2023 n/a as entity was formed...,ticklers 1 or more days past due,2025-07-07,None,WILLITTS S. MENDONCA
1685,ZEITERION THEATRE INC,Management Prepared Interim F/S,,,2025-03-30,2025-05-14,54,Quarterly,Mgmt prepared FYE-12/31/24 financials received...,ticklers 1 or more days past due,2025-07-07,CAMERON J. ETTER,WILLITTS S. MENDONCA
1686,ZEITERION THEATRE INC,Corporate Tax Return,,,2024-12-31,2025-04-30,68,Annually,,ticklers 1 or more days past due,2025-07-07,CAMERON J. ETTER,WILLITTS S. MENDONCA
1687,ZENITH VENTURES LLC,Corporate Tax Return,,,2024-12-31,2025-04-30,68,Annually,,ticklers 1 or more days past due,2025-07-07,JOSHUA A. CAMARA,JOSHUA A. CAMARA


In [81]:
ticklers_coming_due_365 = cleaned_dict['ticklers_coming_due_365'].copy()
ticklers_coming_due_365

,customer_name,item_name,required_value,actual_value,period_date,due_date,days_past_due,interval,comments,report_type,report_date,Loan Officer,Deposit Officer
0,1106 NORTH MAIN STREET LLC,Corporate Tax Return,,,2025-12-31,2026-04-30,,Annually,,ticklers coming due within 365 days,2025-07-07,AN T. LE,AN T. LE
1,"113 BELLINGHAM, LLC",Rent Roll,,,2025-12-31,2026-03-31,,Annually,,ticklers coming due within 365 days,2025-07-07,LAURA A. STACK,LAURA A. STACK
2,"113 BELLINGHAM, LLC",Corporate Tax Return,,,2025-12-31,2026-04-30,,Annually,,ticklers coming due within 365 days,2025-07-07,LAURA A. STACK,LAURA A. STACK
3,117 METRO LLC,CPA Prepared F/S,,,2025-12-31,2026-04-30,,Annually,,ticklers coming due within 365 days,2025-07-07,NaN,NaN
4,12 VENTURA DRIVE REALTY TRUST,Corporate Tax Return,,,2025-12-31,2026-04-30,,Annually,,ticklers coming due within 365 days,2025-07-07,ANDREW RODRIGUES,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2544,ZEITERION THEATRE INC,Corporate Tax Return,,,2025-12-31,2026-04-30,,Annually,,ticklers coming due within 365 days,2025-07-07,CAMERON J. ETTER,WILLITTS S. MENDONCA
2545,ZEITERION THEATRE INC,CPA Prepared F/S,,,2025-12-31,2026-04-30,,Annually,,ticklers coming due within 365 days,2025-07-07,CAMERON J. ETTER,WILLITTS S. MENDONCA
2546,ZENITH VENTURES LLC,Corporate Tax Return,,,2025-12-31,2026-04-30,,Annually,,ticklers coming due within 365 days,2025-07-07,JOSHUA A. CAMARA,JOSHUA A. CAMARA
2547,ZI ZHANG,Personal Financial Statement,,,2025-08-31,2025-09-30,,Annually,,ticklers coming due within 365 days,2025-07-07,NaN,NaN


In [82]:
covenants_past_due = cleaned_dict['covenants_past_due'].copy()
covenants_past_due

,customer_name,item_name,required_value,actual_value,period_date,due_date,days_past_due,interval,comments,report_type,report_date,Loan Officer,Deposit Officer
0,1276 BALD HILL RD LLC,Post-Distribution DSC,1.0,,2024-12-31,2025-04-30,68,Annually,,covenants 1 or more days past due,2025-07-07,KEVIN M. MCCARTHY,KEVIN M. MCCARTHY
1,1276 BALD HILL RD LLC,Pre-Distribution DSC,1.25,,2024-12-31,2025-04-30,68,Annually,,covenants 1 or more days past due,2025-07-07,KEVIN M. MCCARTHY,KEVIN M. MCCARTHY
2,184 ORNE STREET LLC,Investment Property NOI/TDS,1.2,,2024-12-31,2025-04-30,68,Annually,,covenants 1 or more days past due,2025-07-07,WILLITTS S. MENDONCA,WILLITTS S. MENDONCA
3,189 CS PROPERTY LLC,Pre-Distribution DSC,1.25,,2024-12-31,2025-04-30,68,Annually,Post distribution,covenants 1 or more days past due,2025-07-07,CAMERON J. ETTER,KEVIN M. MCCARTHY
4,189 CS PROPERTY LLC,Post-Distribution DSC,1.00,,2024-12-31,2025-04-30,68,Annually,Pre-distribution,covenants 1 or more days past due,2025-07-07,CAMERON J. ETTER,KEVIN M. MCCARTHY
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,WESTPORT TOWNE HOSPITALITY LLC,C&I OCF/TDS,1.20,,2024-12-31,2025-04-30,68,Annually,,covenants 1 or more days past due,2025-07-07,ROGER A. CABRAL,ROGER A. CABRAL
380,WOODFORD REALTY TRUST II,Investment Property NOI/TDS,1.20,,2024-12-31,2025-04-30,68,Annually,,covenants 1 or more days past due,2025-07-07,ROGER A. CABRAL,ROGER A. CABRAL
381,WOODLAWN COMMONS LLC,Investment Property NOI/TDS,1.2,,2024-12-31,2025-04-30,68,Annually,,covenants 1 or more days past due,2025-07-07,DAMON T. ARPIN,DAMON T. ARPIN
382,WW CRANBERRY HIGHWAY INC.,C&I OCF/TDS,1.20,,2024-12-31,2025-04-30,68,Annually,,covenants 1 or more days past due,2025-07-07,CHRISTINE G. PAIVA,ROGER A. CABRAL


In [83]:
covenants_coming_due_365 = cleaned_dict['covenants_coming_due_365'].copy()
covenants_coming_due_365

,customer_name,item_name,required_value,actual_value,period_date,due_date,days_past_due,interval,comments,report_type,report_date,Loan Officer,Deposit Officer
0,12 VENTURA DRIVE REALTY TRUST,LOC 30 Day Clean-up or Clean Down,30.00,,2025-12-31,2026-04-30,,,,covenants coming due within the next 365 days,2025-07-07,ANDREW RODRIGUES,None
1,1276 BALD HILL RD LLC,Maximum LTV of XXX%,70.00,,2025-12-31,2026-02-14,,,,covenants coming due within the next 365 days,2025-07-07,KEVIN M. MCCARTHY,KEVIN M. MCCARTHY
2,1276 BALD HILL RD LLC,Pre-Distribution DSC,1.25,,2025-12-31,2026-04-30,,,,covenants coming due within the next 365 days,2025-07-07,KEVIN M. MCCARTHY,KEVIN M. MCCARTHY
3,1276 BALD HILL RD LLC,Post-Distribution DSC,1.00,,2025-12-31,2026-04-30,,,,covenants coming due within the next 365 days,2025-07-07,KEVIN M. MCCARTHY,KEVIN M. MCCARTHY
4,136 HUTTLESTON AVENUE REALTY TRUST,Maximum LTV of XXX%,85.00,,2025-12-31,2026-04-30,,,,covenants coming due within the next 365 days,2025-07-07,ANDREW RODRIGUES,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,WYCHMERE HARBOR FUNCTIONS LP,Pre-Distribution DSC,1.25,,2025-12-31,2026-04-30,,,,covenants coming due within the next 365 days,2025-07-07,JEFFREY M. VIALL,None
586,WYCHMERE HARBOR FUNCTIONS LP,Maximum LTV of XXX%,65.00,,2025-12-31,2026-04-30,,,,covenants coming due within the next 365 days,2025-07-07,JEFFREY M. VIALL,None
587,WYCHMERE HARBOR FUNCTIONS LP,Pre-Distribution DSC,1.25,,2024-12-31,2025-10-15,,,,covenants coming due within the next 365 days,2025-07-07,JEFFREY M. VIALL,None
588,WYCHMERE HARBOR FUNCTIONS LP,Maximum LTV of XXX%,65.00,,2024-12-31,2025-10-15,,,,covenants coming due within the next 365 days,2025-07-07,JEFFREY M. VIALL,None


In [84]:

covenants_in_default = cleaned_dict['covenants_in_default'].copy()
covenants_in_default

,customer_name,item_name,required_value,actual_value,period_date,due_date,days_past_due,interval,comments,report_type,report_date,Loan Officer,Deposit Officer
0,59-65 70 BLACKSTONE AVENUE REALTY LLC,Investment Property NOI/TDS,1.20,.36,2024-12-31,2025-04-30,,Annually,,covenants 1 or more days in default,2025-07-07,MARK A. BORKMAN,MARK A. BORKMAN
1,A & L PLUMBING INC,LOC 30 Day Clean-up or Clean Down,0,80000,2024-12-31,2025-04-30,,Annually,,covenants 1 or more days in default,2025-07-07,ANDREW J. OMER,ANDREW J. OMER
2,ABC TESTING A LONGER NAME COMPANY,Post-Distribution DSC,1.25,1.1,2024-12-31,2025-03-01,,One-Time,,covenants 1 or more days in default,2025-07-07,CHRISTINE G. PAIVA,None
3,CENTREDALE REVIVAL LLC,C&I OCF/TDS combined with Realty,1.25,-0.55,2022-12-31,2023-04-30,,Annually,,covenants 1 or more days in default,2025-07-07,PETER ST JEAN,PETER ST JEAN
4,CURRY WATERPROOFING & MASONRY RESTORATION INC,LOC 30 Day Clean-up or Clean Down,0,33552.44,2022-12-31,2023-02-14,,Annually,,covenants 1 or more days in default,2025-07-07,SBLC LOAN OFFICER,FRANK P. WILHELM
5,EAST COAST SHED INC,LOC 30 Day Clean-up or Clean Down,30,27,2020-12-31,2021-04-30,,Annually,,covenants 1 or more days in default,2025-07-07,SBLC LOAN OFFICER,GEORGE J. MENDROS
6,HA HOSPITALITY GROUP LLC,C&I OCF/TDS,1.25,0.97,2022-12-31,2023-09-20,,Annually,,covenants 1 or more days in default,2025-07-07,PETER ST JEAN,PETER ST JEAN
7,JNK REALTY LLC,Investment Property NOI/TDS,1.20,0.87,2023-12-31,2024-04-29,,Annually,,covenants 1 or more days in default,2025-07-07,ROGER A. CABRAL,None
8,M.A.G. IRRIGATION INC,LOC 30 Day Clean-up or Clean Down,30,0,2022-12-31,2023-05-01,,Annually,,covenants 1 or more days in default,2025-07-07,BRANDON CANNATA,KAITLYN M. SILVA
9,SPECTRUM THERMAL PROCESSING LLC,C&I OCF/TDS,1.2,0.62,2022-12-31,2023-02-14,,Annually,,covenants 1 or more days in default,2025-07-07,JERMAINE R. MILLER,LAURA A. STACK
